In [ ]:
import pandas as pd
import numpy as np
import datetime
import time
from datetime import datetime
import requests
import re
from lxml import html
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

### Calculate the start and end dates a week before the tweet to get information about the user's activity a week before the tweet

In [ ]:
def before_week(mention_time):
    date = mention_time.split(' ')[0]
    for_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    start_before = for_date + datetime.timedelta(days=-10)
    end_before = for_date + datetime.timedelta(days=-4)
    return start_before, end_before

### Calculate the start and end dates within a week the tweet to get information about the user's activity within a week the tweet

In [ ]:
def during_week(mention_time):
    date = mention_time.split(' ')[0]
    for_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    start_dur = for_date + datetime.timedelta(days=-3)
    end_dur = for_date + datetime.timedelta(days=3)
    return start_dur, end_dur

### Calculate the start and end dates a week after the tweet to get information about the user's activity a week after the tweet

In [ ]:
def after_week(mention_time):
    date = mention_time.split(' ')[0]
    for_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    start_after = for_date + datetime.timedelta(days=4)
    end_after = for_date + datetime.timedelta(days=10)
    return start_after, end_after

### Read csv file

In [ ]:
tweets = pd.read_csv('../../data/Tweets/English_tweets.csv',
                     index_col=False)

### Get Activity Descriptions

In [ ]:
user_activity = []
activity_des = []
activity_URL = []
total_activity = []
import datetime
import time
for mention_time, sponsor_username in tqdm(
        zip(tweets.iloc[:, 2], tweets.iloc[:, 7])):
    # Here to change function to get different time duration activity
    after_times = after_week(mention_time)
    start = after_times[0]
    end = after_times[1]
    URL = "https://github.com/{0}?tab=overview&from={1}&to={2}".format(
        sponsor_username, start, end)
    activity_URL.append(URL)
    page = requests.get(URL)
    activity = page.text
    all_activity = BeautifulSoup(activity, 'lxml')
    get_all = all_activity.find_all('div', class_='TimelineItem-body')
    for item in get_all:
        get_span = item.find('span')
        get_h4 = item.find('h4')
        if str(get_h4) == 'None':
            span_text = get_span.text.strip()
            text = " ".join(span_text.split())
            activity_des.append(text)
        else:
            h4_text = get_h4.text.strip()
            texts = " ".join(h4_text.split())
            activity_des.append(texts)
    user_activity.append(activity_des)
    total_activity.append(len(activity_des))
    activity_des = []
    time.sleep(2)

### Save the user's activity during the corresponding time period, as well as the corresponding links

In [ ]:
tweets['week_after'] = user_activity
tweets['week_after_url'] = activity_URL

### Save as csv file

In [ ]:
tweets.to_csv('../../data/Contribution_activities/week_after.csv',
              index=False)